# Static electromagnetic fields

$
\newcommand{\Ec}{\mathcal{E}}
\newcommand{\Mc}{\mathcal{M}}
\newcommand{\Hc}{\mathcal{H}}
\newcommand{\Bc}{\mathcal{B}}
\newcommand{\Jc}{\mathcal{J}}
\newcommand{\Dc}{\mathcal{D}}
\newcommand{\veps}{\varepsilon}
\newcommand{\og}{\omega}
\newcommand{\Om}{\Omega}
\newcommand{\ii}{\imath}
\newcommand{\Jcl}{J_{\pi/2}}
\renewcommand{\d}{\partial}
\newcommand{\R}{\mathbb{R}}
\DeclareMathOperator{\rot}{rot}
\DeclareMathOperator{\grad}{grad}
\DeclareMathOperator{\curl}{curl}
\DeclareMathOperator{\dive}{div}
\newcommand{\Ho}{\smash[t]{\mathring{H}}}
$
According to Maxwell equations, a time-varying electric field $\Ec$ creates a magnetic field $\Hc$, and vice versa. Since time variations in one field cause  coupling to the other field, we anticipate that when nothing is changing in time, the field equations should decouple.

To see this in more detail, recall the Maxwell system 
\begin{align*}
    \d_t \Bc + \curl \Ec & = 0
    \\
    \d_t \Dc - \curl \Hc & = -\Jc 
    \\
    \dive \Bc & = 0
    \\
    \dive \Dc & = \rho,
\end{align*}
where $\Dc, \Bc, \Jc,$ and $\rho$ denotes the displacement current, the magnetic flux density, the current density, and the charge density, respectively. When the current $\Jc$ and charge density $\rho$  are time independent, it is natural to seek solutions that are also time-independent.  Omitting the $\d_t$-terms from the Maxwell system, 
we  find that $\Ec, \Dc$  decouple from $\Hc, \Bc$. Namely, $\Ec$ and $\Dc$ satisfy the **electrostatic system** 

\begin{align*}
    \curl \Ec & = 0, 
    \\
    \dive \Dc & = \rho, 
\end{align*}

while $\Hc$ and $\Bc$ satisfy the **magnetostatic system** 

\begin{align*}
    \curl \Hc & = \Jc, 
    \\
    \dive \Bc & = 0.
\end{align*}

The electrostatic system must be complemented with constitutive laws connecting $\Dc$ and $\Ec$. Similarly, the magnetostatic system must be augmented with constitutive laws connecting $\Bc$ and $\Hc$. These systems give a variety of stationary Maxwell solutions in $\R^3$.

## Electrostatics

$\newcommand{\Ec}{\mathcal{E}}
\newcommand{\Mc}{\mathcal{M}}
\newcommand{\Hc}{\mathcal{H}}
\newcommand{\Bc}{\mathcal{B}}
\newcommand{\Jc}{\mathcal{J}}
\newcommand{\Dc}{\mathcal{D}}
$

Since any curl-free smooth vector field $\Ec$ in $\R^3$ must be a gradient, the first equation of electrostatics implies the existence of an *electric potential* $\Phi$ such that 
$$
\Ec = -\grad \Phi.
$$
In a linear medium, since $\Dc = \veps \Ec$ (where $\veps$ is the electric permittivity), the second equation of electrostatics determines the electric potential by 
$$
-\dive (\veps \grad \Phi) = \rho, 
$$
the Poisson equation. Since we have already extensively studied this equation, let us skip to the next system describing magnetostatics. 

## Magnetostatics

Since any divergence-free smooth $\Bc$ in $\R^3$  must be a curl, the second  equation of magnetostatics, $\dive \Bc=0$,  implies that there is a *magnetic vector potential* $A$ such that 
$$
\Bc = \curl A.
$$
If the medium admits  the constitutive relation  $\Bc = \mu \Hc$,  where $\mu$ is the magnetic permeability, then the first equation of magnetostatics, $\curl \Hc = \Jc$,   leads to the following equation for $A$:
$$
\curl \mu^{-1} \curl A = \Jc.
$$
Note that the vector potential is not unique. We have seen how a gauge constraint makes $A$ unique and results in a wellposed mixed system. This notebook focuses on practical issues in computing magnetostatic fields through two selected examples (cf. ngsolve [i-tutorial](https://ngsolve.org/docu/nightly/i-tutorials/unit-2.4-Maxwell/Maxwell.html) and [wta](http://127.0.0.1:8888/lab?token=2b65da817d6f054c13930b9ca6f7c1892b90fef3e4dd0a87)). 

In [ ]:
from netgen.occ import Cylinder, Box, X, Y, Z, OCCGeometry, Glue
from netgen.occ import Pnt, Edge, Segment, BezierCurve, Circle
from netgen.occ import Wire, Face, Pipe
from netgen.meshing import meshsize
from ngsolve.webgui import Draw
from ngsolve import Mesh
import ngsolve as ng
from math import pi
from ngsolve import grad, curl, dx, ds

## Example 1:  Magnetic field around a magnet

The magnetic field generated by a stationary ferromagnet is a typical example where magnetostatics apply (since nothing is varying in time).  For this example,  we must change  the above-mentioned constitutive equation $\Bc = \mu \Hc$. The reason is that when the atomic constituents of a medium has [magnetic dipoles pointed in the same direction](https://en.wikipedia.org/wiki/Ferromagnetism), 
such as in a ferromagnet, they create a nontrivial **magnetization** field $\Mc$, which must be accounted for in a [revised constitutive law](https://en.wikipedia.org/wiki/Magnetization),
$$ \newcommand{\Mc}{\mathcal{M}}
\newcommand{\Hc}{\mathcal{H}}
\Bc = \mu ( \Hc + \Mc),
$$
where $\mu$ is again the magnetic permeability.

*<font color=blue>The goal, in this example, is to compute the $\Bc$-field created by a **bar magnet** placed within an electromagnetically isolating box in air.</font>* Here is the geometry:

In [ ]:
box = Box(Pnt(-3, -3, -3), Pnt(3, 3, 3)).bc('outer')
magnet = Cylinder(Pnt(-1,0,0), X, r=0.3, h=2)
magnet.faces.col=(0.3, 0.3, 0.4)
magnet.mat('magnet')
air = box - magnet
air.mat('air')

ma = Glue([magnet, air])
geo = OCCGeometry(ma)
mesh = Mesh(geo.GenerateMesh(maxh=2)).Curve(3)
Draw(ma, clipping={"y":0, "z":-1});

The known parameters in this problem are $\Mc$ and $\mu$. In SI units,  [Values](https://en.wikipedia.org/wiki/Magnet) of magnetization of iron are around $10^6$ A/m. The magnetic permeability  is usually given in terms of the vacuum permeability  $\mu_0 = 4\pi \times 10^{-7}$ as 
$$
\mu = \mu_r \mu_0,
$$
where $\mu_r$ is the relative magnetic permeability.  The relative magnetic permeability of iron [varies](https://en.wikipedia.org/wiki/Permeability_(electromagnetism)) significantly: for our model problem, we will fix its value to  $5000$ H/m.

Accordingly, within the bar magnet, which occupies the computational subdomain we denote by  $\Omega_\text{magnet}$, the magnetization $\Mc$ is modeled as a constant nonzero vector set below. Outside of the magnet, $\Mc$ vanishes:
$$
\Mc = \left\{
\begin{aligned}
& (10^6, 0, 0) && \text{ in } \Omega_\text{magnet},
\\
& (0, 0, 0) && \text{ in } \Omega \setminus \Omega_\text{magnet}.
\end{aligned}
\right.
$$
The relative magnetic permeability is set by 
$$
\mu_r
= \left\{
\begin{aligned}
& 5000 && \text{ in } \Omega_\text{magnet},
\\
& 1 && \text{ in } \Omega \setminus \Omega_\text{magnet}.
\end{aligned}
\right.
$$

In [ ]:
M = mesh.MaterialCF({'magnet': (1e6,0,0), 'air': (0,0,0)})
mu0 = 4*pi*1e-7
mur = mesh.MaterialCF({'magnet': 5000}, default=1)

There are no applied currents in the system, i.e., 
$$
\Jc = 0.
$$
Hence $\Mc$ serves as the source for solving the magnetostatic field.  As before, since $\dive \Bc = 0,$ introducing a magnetic vector potential $A$ such that $\Bc = \curl A$, the equations 
$$
\curl \Hc = 0, \qquad \Bc = \mu( \Hc + \Mc)
$$
imply that $\curl (\mu^{-1} \Bc - \Mc)=0$. Thus we have the following boundary value problem for $A$:  Given $\Mc$ and $\mu$,  find $A$ satisfying
\begin{align*}
\curl \mu^{-1} \curl A & = \curl \Mc && \text{ in } \Omega, \\
 A \times n & = 0 && \text{ on } \d\Omega.
\end{align*}

Note that the boundary condition $A \times n =0$ implies that $n \cdot \curl A = n \cdot B = 0,$ so it models an isolating box that confines the magnetic field to the interior of the box.

Instead of computing with large material coefficients like $\mu^{-1}$, it's nicer to compute with non-dimensional numbers like $\mu_r$. Hence, multiplying through by the universal constant $\mu_0,$ we revise the problem:

<font color=blue>Given $\Mc$ and $\mu$, find the magnetostatic potential $A$ satisfying
\begin{align*}
\curl \mu_r^{-1} \curl A & = \curl (\mu_0\Mc) && \text{ in } \Omega, \\
 A \times n & = 0 && \text{ on } \d\Omega.
\end{align*}
</font>


At this point, an issue worth noting is that since $\Mc$ is discontinuous, $\curl (\mu_0\Mc)$ can only be interpreted in the distributional sense. In other words, 
the above equation $\curl \mu^{-1} \curl A  = \curl (\mu_0\Mc)$ is to be rigorously considered as a *distributional equation*. Using the definition of the distributional curl,
$$
\int_\Omega \mu_r^{-1} \curl A \cdot \curl \varphi = \int_\Omega \mu_0\Mc \cdot \curl \varphi
$$
for all $\varphi$ in the space of smooth Schwartz test functions $\Dc(\Omega)^3.$ Since $\Dc(\Omega)^3$ is dense in $\Ho(\curl, \Omega)$, we obtain the following weak formulation:
Find $A \in \Ho(\curl,\Omega)$ satisfying 
\begin{align*}
\int_\Omega  \mu_r^{-1} \curl A \cdot \curl v  = \int_\Omega \mu_0\Mc \cdot \curl v
\end{align*}
for all $v \in \Ho(\curl,\Omega)$.   This reformulation allows us to avoid the tricky issue of numerically approximating   the surface field contribution of the distribution $\curl \Mc$.

Now we must face another issue. The above weak formulation is not uniquely solvable. Indeed, if you find a solution $A,$ then $A+ \grad \psi$ for any $\psi \in \Ho^1(\Omega)$ must also solve the same boundary value problem. This is a reflection of the fact that magnetic vector potentials are not unique. One technique to recover uniqueness, as seen in the theory lectures,  is to impose an *additional gauge constraint* that $A$ must be  orthogonal to  $\grad \Ho^1(\Omega)$. This is the second equation of the revised weak formulation we state next. The imposition of such a constraint results in the introduction of a Lagrange multiplier $\phi\in \Ho^1(\Omega)$ into the other equation of the weak form.

<font color=blue>Find $A \in \Ho(\curl,\Omega)$ and $\phi \in \Ho^1(\Omega)$ satisfying \begin{align*}
\int_\Omega  \mu_r^{-1} \curl A \cdot \curl v + \int_\Omega \grad \phi \cdot v 
&  = \int_\Omega \mu_0\Mc \cdot \curl v\\
\int_\Omega A \cdot \grad \psi & = 0 
\end{align*}
for all $v \in \Ho(\curl,\Omega)$ and $\psi \in \Ho^1(\Omega)$. 
</font>

<div class="alert alert-info">
    <b><font color=red>Exercise I:</font></b>  
    Prove that any $\phi\in \Ho^1(\Omega)$ solving  the above formulation must be zero.
</div>

By verifying the conditions of Brezzi theory, we have seen that this weak formulation is uniquely solvable. Its natural discretization, using Nedelec elements for $A$ and Lagrange elements for $\phi$, fitting the finite element de Rham subcomplex, yields optimal orders of convergence.

We now proceed to implement this method and visualize the resulting magnetic flux density $\Bc$.

In [ ]:
N = ng.HCurl(mesh, order=0, type1=True, dirichlet='outer')
V = ng.H1(mesh, order=1, dirichlet='outer')
NV = ng.FESpace([N, V])      
(A, p), (v, q) = NV.TnT()

a = ng.BilinearForm(NV)
a += (1.0/mur)*curl(A)*curl(v)*dx + grad(p)*v*dx
a += grad(q)*A*dx 

f = ng.LinearForm(NV)
f += mu0 * M * curl(v) * dx("magnet")

with ng.TaskManager():
    a.Assemble()
    f.Assemble()

In [ ]:
Ap = ng.GridFunction(NV)
try:
    Ap.vec.data = a.mat.Inverse(NV.FreeDofs(), inverse='umfpack') * f.vec
except:  # do without a sparse direct solver:
    from ngsolve.solvers import GMRes
    with ng.TaskManager():
        Ap.vec[:] = 0
        c = ng.BilinearForm(NV)
        c += ((1.0/mur)*curl(A)*curl(v) + A*v + grad(p)*grad(q) + p*q) * dx
        c.Assemble()
        cinv = c.mat.Inverse(NV.FreeDofs(), inverse='sparsecholesky')
        GMRes(A=a.mat, pre=cinv, b=f.vec, x=Ap.vec, tol=1e-13, printrates=False, maxsteps=1000)

Calculating the curl of this computed $A$, we obtain a visualization of $\Bc$.

In [ ]:
B = curl(Ap.components[0])

Draw(B, mesh, 'B-field', draw_surf=False, vectors={'grid_size':50}, clipping = {'y':0, 'z': -1, 'function':False});

In accordance with our intuition from elementary physics, the computed results show that the $\Bc$-field appears to emanate out of one side (the "north" pole of the magnet) and loop back into the other side (the "south" pole).

## Example 2: Magnetic field due to current in a coil


This second example is also a three-dimensional example, but it is considerably more computationally expensive.  The scenario is as follows. A current through a coil creates a magnetic field, per Maxwell equations. When the current is stationary, we can use the equations of magnetostatics to compute the magnetic field.

The first task is to make the coil geometry. The region occupying the coil $\Om_\text{coil}$ is shown clipped below.

In [ ]:
cyl = Cylinder(Pnt(0,0,0), Z, r=0.01, h=0.03).faces[0]
heli = Edge(Segment((0,0), (12*pi, 0.03)), cyl)
ps = heli.start
vs = heli.start_tangent
pe = heli.end
ve = heli.end_tangent

e1 = Segment((0,0,-0.03), (0,0,-0.01))
c1 = BezierCurve( [(0,0,-0.01), (0,0,0), ps-vs, ps])
e2 = Segment((0,0,0.04), (0,0,0.06))
c2 = BezierCurve( [pe, pe+ve, (0,0,0.03), (0,0,0.04)])
spiral = Wire([e1, c1, heli, c2, e2])
circ = Face(Wire([Circle((0,0,-0.03), Z, 0.001)]))
coil = Pipe(spiral, circ)
coil.faces.maxh=0.2
coil.faces.name="coilbnd"
coil.faces.Max(Z).name="in"
coil.faces.Min(Z).name="out"
coil.mat("coil")
Draw(coil, euler_angles=(-45, 35, 20));

Next, we enclose the coil in a box of air, followed by meshing the resulting domain,
$$
\Om = \Om_{\text{air}} \cup \Om_{\text{coil}}.
$$
We track the air and coil subdomains by naming them distinctly. 

In [ ]:
box = Box((-0.04,-0.04,-0.03), (0.04,0.04,0.06))
box.faces.name = 'outer'
air = box-coil
air.mat('air');

In [ ]:
geo = OCCGeometry(Glue([coil,air]))
with ng.TaskManager():
    msh = ng.Mesh(geo.GenerateMesh(meshsize.coarse)).Curve(3)   

In [ ]:
Draw(msh, clipping={"y":1, "z":0, "dist":0.012}, euler_angles=(-140, -50, 0));

In addition to this geometry, the only other pieces of information we are supplied with is that the coil is made of copper and  that a potential difference of 1000 V is applied across the two ends of the coil.

Given these bits of information, *<font color=blue>the problem is to compute the magnetic field created by the current in the coil due to the voltage difference.</font>*

Returning to the system of partial differential equations for the magnetic flux density $\Bc$, 
\begin{align*}
    \curl \Hc & = \Jc, 
    &
    \dive \Bc & = 0.
\end{align*}
Combined with the linear material constitutive law,  $\Bc = \mu \Hc$, as before, introducing a magnetic vector potential $A$ such that $\Bc = \curl A$, the task is to find an $A$ solving

$$
\curl \mu^{-1} \curl A = \Jc.
$$

The system is driven by the current density $\Jc$. However, we are not given $\Jc$. 
So we need to first discuss how  we can compute the current density $\Jc$ based off the given potential difference information.

#### Computing the source current 

Let us begin by recalling that since nothing is varying in time, by the equations of electrostatics mentioned at the beginning of this notebook, there exists an electric potential $\Phi$ such that $\Ec = -\grad \Phi$. The current density is given by Ohm's law,

$$
\Jc = \sigma \Ec = -\sigma \grad \Phi.
$$

Recalling that the coil is made of copper, note that the [conductivity](https://en.wikipedia.org/wiki/Electrical_resistivity_and_conductivity) of copper is  large, $\sigma \approx 5.96 \times 10^7$, while that of air is negligible, varying between $10^{-15}$ to $10^{-9}$. Thus the current density $\Jc$ may be modeled as zero in the air region. To find $\Jc$ in the coil, note that the Maxwell equation $\curl \Hc = -\Jc$ implies $\dive \Jc =0$ in $\Om$. Having agreed that $\Jc\equiv 0$ in $\Om_{\text{air}}$, a consistent boundary condition at the air-copper interface would be $\Jc \cdot n =0$. Using it, we may restrict the problem of computing $\Jc$ to $\Om_{\text{coil}}$. Namely,  $\Jc = \sigma \Ec = -\sigma \grad \Phi$,  and the electrostatic potential $\Phi$ solves
\begin{align*}
\dive(\sigma \grad \Phi) & = 0 && \text{ in }\Om_{\text{coil}},
\\
\Phi & = 10^3  && \text{ on }\d\Om_{\text{in}},
\\
\Phi & = 0  && \text{ on }\d\Om_{\text{out}},
\\
\sigma\frac{\d \Phi}{\d n} & = 0 && \text{ on }\d\Om_{\text{coil}} \setminus \Om_{\text{out}},
\end{align*}
where the current inlet and outlet faces of the coil are denoted by 
$\d\Om_{\text{in}}$ and $\d\Om_{\text{out}},$ respectively, and were named `in` and `out` in the meshing code above. Note that since $\sigma$, being a constant within the coil, can be canceled off from the first and last equations of the boundary value problem above. We first solve this  electrostatic problem, using Lagrange elements as usual, restricted to the coil subdomain, below.

In [ ]:
p=3
V = ng.H1(msh, order=p, definedon='coil', dirichlet='in|out')  # FE space on just one subdomain
phi, psi = V.TnT()
Phi = ng.GridFunction(V)
Phi.Set(1000, definedon=msh.Boundaries('in'))

In [ ]:
with ng.TaskManager():
    a = ng.BilinearForm(grad(phi)*grad(psi)*dx).Assemble().mat
    ainv = a.Inverse(freedofs=V.FreeDofs(), inverse="sparsecholesky")
    f = Phi.vec.CreateVector()
    f.data = -a * Phi.vec
    Phi.vec.data += ainv * f

In [ ]:
Draw(Phi, draw_vol=False, clipping={"y":1, "z":0, "dist":0.012}, euler_angles=(-140, -50, 0));

Now we are finally ready to compute the input data required to solve the magnetostatic field problem, namely the  current density source $\Jc = -\sigma \grad\Phi$, using the gradient of the above-shown $\Phi$. 

This then goes into computing the right hand side of 
$
\curl \mu^{-1} \curl A = \Jc.
$
Note that the magnetic permeability of copper and air are about the same as in vacuum, so we may as well simplify and model $\mu$ as the constant $\mu_0$ and multiply the above equation through by it. This yields  the equation we shall now solve, namely 
$$
\curl \curl A = F,
$$
with 
$$
F = -\mu \sigma \grad \Phi.
$$
Let us compute and visualize this source. 

In [ ]:
sigma = 5.96e7
mu0 = 4*pi*1e-7
F = -sigma * mu0 * grad(Phi)
Draw(F, msh, draw_surf=False, min=-1000, max=1000, vectors = {'grid_size':500}, clipping={"y":1, "z":0, "dist":0.0}, euler_angles=(-140, -50, 0));

In this plot, you will need to zoom in closer to the coil to appreciate the directions (in various clipped spots) of source vector field representing the current flow that we just computed. 

#### Computing the magnetic field
Now that we have the right hand side prepared, we can solve for $A$. Of course, we must deal with the same non-uniqueness problem we encountered in Example 1. This time we illustrate another technique to overcome the problem. Instead of orthogonalizing $A$ against gradients, we use an ngsolve facility to create an $H(\curl)$-conforming space from which most gradient shape functions have been removed. This together with the addition of a small term that prevents the system from having a null space is a reasonable practical technique to overcome the nonuniqueness. Even if  it may not be as mathematically satisfying as the previous foolproof mixed method technique of Example 1, it does have the advantage that we can solve a much larger sized problem with a simple iterative solver, namely conjugate gradients, without huge memory demands.

In [ ]:
N = ng.HCurl(msh, order=2,
             nograds=True, # remove most gradient fields from Nedelec space
             dirichlet='outer')  
A, v  = N.TnT()
d = 1e-6   # add a small positive perturbation to get uniqueness
a = ng.BilinearForm(curl(A)*curl(v)*dx + d*A*v*dx)
f = ng.LinearForm(F*v*dx('coil'))
with ng.TaskManager():
    pre = ng.Preconditioner(a, 'bddc')
    a.Assemble()
    f.Assemble()

In [ ]:
inv = ng.CGSolver(a.mat, pre, printrates=True)
Ah = ng.GridFunction(N)
with ng.TaskManager():
    Ah.vec.data = inv * f.vec

In [ ]:
Draw (curl(Ah), msh, draw_surf=False,  min=0, max=100, vectors ={'grid_size':100}, clipping={"y":1, "z":0}, euler_angles=(-70, 0, 0));

A close examination of this vector field will show you the similarities this $\Bc$-field has with the field around the permanent magnet of Example 1. The magnetic field loops around the coil much the same way as it did through the magnet.


<hr>
    
    
$\ll$ [Table Of Contents](./0_INDEX.ipynb) <br>
$\ll$ [Jay Gopalakrishnan](http://web.pdx.edu/~gjay/)